# Prompt Template Engine for Wearable Insights

This notebook demonstrates how to build and test prompt templates for generating personalized insights from wearable data. We'll use the processed data from the previous notebook to create various prompt templates and preview the outputs.

In [ ]:
# Import necessary libraries
import sys
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Add the src directory to the path so we can import our modules
sys.path.append('../')
from src.insight_prompt_builder import InsightPromptBuilder
from src.llm_engine import LLMEngine

# Set up plotting
%matplotlib inline

## 1. Load Processed Data

First, let's load the processed data from the previous notebook.

In [ ]:
# Load combined features
processed_dir = '../data/processed'
with open(os.path.join(processed_dir, 'combined_features.json'), 'r') as f:
    combined_features = json.load(f)

# Load user goals
with open(os.path.join(processed_dir, 'user_goals.json'), 'r') as f:
    user_goals = json.load(f)

print(f"Loaded data for {len(combined_features)} days")
print(f"User: {user_goals['name']}")

In [ ]:
# Get the most recent day's data
dates = list(combined_features.keys())
dates.sort()
latest_date = dates[-1]
latest_features = combined_features[latest_date]

print(f"Latest data date: {latest_date}")
print(f"Number of features: {len(latest_features)}")

# Display a few key features
key_features = [
    'hrv_rmssd_mean', 'activity_total_steps', 'sleep_total_sleep_hours', 'recovery_score'
]
print("\nKey metrics:")
for feature in key_features:
    if feature in latest_features:
        value = latest_features[feature]
        print(f"- {feature}: {value}")

## 2. Initialize Prompt Builder

Now, let's initialize the prompt builder and explore different prompt templates.

In [ ]:
# Initialize prompt builder
prompt_builder = InsightPromptBuilder()

# List available tones
available_tones = list(prompt_builder.templates.keys())
print(f"Available tones: {available_tones}")

## 3. Build Basic Prompts with Different Tones

Let's create prompts with different tones to see how they affect the output.

In [ ]:
# Extract the user's primary goal for fitness
fitness_goal = next((goal['goal'] for goal in user_goals['primary_goals'] if goal['area'] == 'fitness'), 
                    "improving overall fitness")

print(f"User's fitness goal: {fitness_goal}")

In [ ]:
# Build a coach-tone prompt
coach_prompt = prompt_builder.build_prompt(
    latest_features,
    tone="coach",
    user_goal=fitness_goal,
    time_range="yesterday"
)

print("=== COACH PROMPT ===\n")
print("System prompt:")
print(coach_prompt["system"])
print("\nUser prompt:")
print(coach_prompt["user"])

In [ ]:
# Build a medical-tone prompt
medical_prompt = prompt_builder.build_prompt(
    latest_features,
    tone="medical",
    user_goal=fitness_goal,
    time_range="yesterday"
)

print("=== MEDICAL PROMPT ===\n")
print("System prompt:")
print(medical_prompt["system"])
print("\nUser prompt:")
print(medical_prompt["user"])

In [ ]:
# Build a motivational-tone prompt
motivational_prompt = prompt_builder.build_prompt(
    latest_features,
    tone="motivational",
    user_goal=fitness_goal,
    time_range="yesterday"
)

print("=== MOTIVATIONAL PROMPT ===\n")
print("System prompt:")
print(motivational_prompt["system"])
print("\nUser prompt:")
print(motivational_prompt["user"])

## 4. Build Focused Prompts

Now, let's create prompts focused on specific areas like sleep, recovery, or activity.

In [ ]:
# Extract the user's sleep goal
sleep_goal = next((goal['goal'] for goal in user_goals['primary_goals'] if goal['area'] == 'sleep'), 
                  "improving sleep quality")

# Build a sleep-focused prompt
sleep_prompt = prompt_builder.build_focused_prompt(
    latest_features,
    focus_area="sleep",
    user_goal=sleep_goal
)

print("=== SLEEP-FOCUSED PROMPT ===\n")
print("System prompt:")
print(sleep_prompt["system"])
print("\nUser prompt:")
print(sleep_prompt["user"])

In [ ]:
# Build a recovery-focused prompt
recovery_prompt = prompt_builder.build_focused_prompt(
    latest_features,
    focus_area="recovery",
    user_goal="optimizing recovery between workouts"
)

print("=== RECOVERY-FOCUSED PROMPT ===\n")
print("System prompt:")
print(recovery_prompt["system"])
print("\nUser prompt:")
print(recovery_prompt["user"])

## 5. Build Comparative Prompts

Let's create prompts that compare current data with previous periods.

In [ ]:
# Get the previous day's data
if len(dates) > 1:
    previous_date = dates[-2]
    previous_features = combined_features[previous_date]
    
    # Build a comparative prompt
    comparative_prompt = prompt_builder.build_comparative_prompt(
        latest_features,
        previous_features,
        tone="coach",
        user_goal=fitness_goal
    )
    
    print("=== COMPARATIVE PROMPT ===\n")
    print("System prompt:")
    print(comparative_prompt["system"])
    print("\nUser prompt:")
    print(comparative_prompt["user"])
else:
    print("Not enough data for comparative analysis.")

## 6. Add Few-Shot Examples

Let's enhance our prompts with few-shot examples to guide the LLM's responses.

In [ ]:
# Define few-shot examples
few_shot_examples = [
    {
        "input": "HRV: RMSSD 65.3, Sleep: 7.2 hours, Steps: 8500, Recovery Score: 75.8",
        "output": "Your HRV is in a good range, indicating balanced stress and recovery. With 7.2 hours of sleep, you're close to the recommended 7-8 hours. Your recovery score of 75.8 suggests you're ready for moderate intensity training today. Consider a zone 2 cardio session or strength training with adequate rest between sets."
    },
    {
        "input": "HRV: RMSSD 45.2, Sleep: 5.8 hours, Steps: 12000, Recovery Score: 58.3",
        "output": "Your HRV is lower than your baseline, indicating increased stress or incomplete recovery. Combined with less than optimal sleep (5.8 hours), your body is showing signs of fatigue. Despite high activity yesterday (12000 steps), your recovery score of 58.3 suggests prioritizing rest today. Consider a light mobility session or yoga instead of intense training."
    }
]

# Add few-shot examples to a prompt
coach_prompt_with_examples = prompt_builder.add_few_shot_examples(coach_prompt, few_shot_examples)

print("=== COACH PROMPT WITH FEW-SHOT EXAMPLES ===\n")
print("System prompt:")
print(coach_prompt_with_examples["system"])
print("\nUser prompt:")
print(coach_prompt_with_examples["user"])

## 7. Create a Custom Prompt Template

Let's create and save a custom prompt template for future use.

In [ ]:
# Define a custom template
custom_system_prompt = """You are an elite endurance coach analyzing biometric data from wearable devices. 
Your expertise is in optimizing training for endurance athletes based on physiological markers.
Focus on providing specific, actionable training recommendations based on HRV, sleep, and recovery metrics.
Include both the 'what' and the 'why' in your advice, explaining the physiological reasoning."""

custom_user_prompt = """Here's my recent biometric data as an endurance athlete:
{data_summary}

My current goal is {user_goal}. Based on these metrics, what specific workout should I do today? 
Please recommend intensity, duration, and type of session, along with the physiological reasoning."""

# Save the custom template
templates_dir = '../outputs'
os.makedirs(templates_dir, exist_ok=True)
templates_path = os.path.join(templates_dir, 'custom_templates.json')

prompt_builder.save_prompt_template(
    "endurance_coach",
    custom_system_prompt,
    custom_user_prompt,
    templates_path
)

print(f"Saved custom template to {templates_path}")

In [ ]:
# Load the custom template
with open(templates_path, 'r') as f:
    custom_templates = json.load(f)

# Create a new prompt builder with the custom templates
custom_prompt_builder = InsightPromptBuilder(templates_path=templates_path)

# Build a prompt using the custom template
endurance_prompt = custom_prompt_builder.build_prompt(
    latest_features,
    tone="endurance_coach",
    user_goal="preparing for a marathon in 8 weeks"
)

print("=== CUSTOM ENDURANCE COACH PROMPT ===\n")
print("System prompt:")
print(endurance_prompt["system"])
print("\nUser prompt:")
print(endurance_prompt["user"])

## 8. Generate Sample Insights (Optional)

If you have an API key configured, you can generate sample insights using the LLM engine.

In [ ]:
# Check if API key is set
api_key = os.environ.get("OPENAI_API_KEY")

if api_key:
    # Initialize LLM engine
    llm_engine = LLMEngine()
    
    # Generate insight using coach prompt
    insight, metadata = llm_engine.generate_insight(coach_prompt)
    
    print("=== GENERATED INSIGHT (COACH TONE) ===\n")
    print(insight)
    print("\nMetadata:")
    print(f"- Model: {metadata.get('model')}")
    print(f"- Tokens: {metadata.get('total_tokens')}")
    print(f"- Latency: {metadata.get('latency_seconds'):.2f} seconds")
    
    # Save the response
    output_path = llm_engine.save_response(
        coach_prompt, 
        insight, 
        metadata, 
        '../outputs',
        'sample_coach_insight.json'
    )
    print(f"\nSaved response to {output_path}")
    
else:
    print("OpenAI API key not set. To generate insights, set the OPENAI_API_KEY environment variable.")
    print("Example: export OPENAI_API_KEY='your-api-key'")

## 9. Compare Different Prompt Strategies

Let's compare the different prompt strategies we've explored.

In [ ]:
# Create a table comparing the different prompt strategies
prompt_strategies = [
    {"Strategy": "Basic Coach", "Description": "General coaching tone with all metrics", "Best For": "Overall fitness guidance"},
    {"Strategy": "Medical Tone", "Description": "Clinical analysis of physiological data", "Best For": "Health-focused insights"},
    {"Strategy": "Motivational", "Description": "Encouraging tone emphasizing progress", "Best For": "Motivation and adherence"},
    {"Strategy": "Sleep Focus", "Description": "Detailed analysis of sleep metrics", "Best For": "Sleep optimization"},
    {"Strategy": "Recovery Focus", "Description": "Analysis of recovery readiness", "Best For": "Training periodization"},
    {"Strategy": "Comparative", "Description": "Comparison with previous period", "Best For": "Trend analysis and progress tracking"},
    {"Strategy": "Few-Shot Examples", "Description": "Guided responses with examples", "Best For": "Consistent, targeted insights"},
    {"Strategy": "Custom Templates", "Description": "Domain-specific expertise", "Best For": "Specialized use cases (e.g., endurance training)"}
]

strategies_df = pd.DataFrame(prompt_strategies)
strategies_df

## Summary

In this notebook, we've explored various prompt engineering strategies for generating insights from wearable data:

1. **Different Tones**: We created prompts with coaching, medical, and motivational tones to address different user needs.

2. **Focused Prompts**: We built prompts that focus on specific areas like sleep or recovery, highlighting the most relevant metrics.

3. **Comparative Analysis**: We created prompts that compare current data with previous periods to identify trends and changes.

4. **Few-Shot Examples**: We enhanced prompts with examples to guide the LLM toward specific response formats.

5. **Custom Templates**: We created and saved custom templates for specialized use cases.

These prompt strategies can be combined and customized to generate highly personalized insights based on user goals, preferences, and the specific metrics available in the wearable data.

In the next notebook, we'll integrate these prompt strategies into a complete LLM pipeline for generating insights.